In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
import numpy as np

In [6]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.2.1'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages.dropna(inplace=True)

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2182, 901, 1893, 1604, 2118, 2230, 3082, 807, 1782, 1928],
 [1765, 4102, 4350, 288, 1054, 4265, 629],
 [4088, 68, 2648, 4562],
 [3535, 712, 444, 1456, 3095, 980],
 [2139, 1054, 3414, 272, 2657, 4452, 1054, 2308, 3380, 4764],
 [3548,
  567,
  4503,
  3904,
  3776,
  3227,
  2527,
  848,
  1783,
  3206,
  509,
  4126,
  2561,
  3988,
  629],
 [1687, 3480, 79, 3835, 3399, 3629, 884, 3493, 3825, 4974, 3247],
 [4495, 3102, 104, 1237, 3074, 2157, 3227, 2800, 3825, 4974, 3247],
 [4254, 301, 2998, 4251, 3326, 1113, 3191, 1386, 3227, 2991],
 [3259, 814, 4681, 243, 4370, 1129, 2411, 2828],
 [4536, 1113, 1925, 2041, 2428, 1520, 1690, 229, 909, 4758, 1934],
 [1456, 2184, 2118, 1113, 3227, 3074],
 [833, 2970, 2738, 2881, 4736, 1927, 1990, 92, 568],
 [2305, 1777, 4029, 4743, 535, 4146, 4753, 3825, 4974, 3247],
 [387, 3091, 3157, 3733, 1448, 3825, 4974, 3247],
 [2950, 4346, 234, 4563, 3033, 2435, 2523, 3961, 3360, 3427],
 [4840, 169, 4102],
 [4180, 1681, 1923, 2044, 3227, 4351, 857, 629],
 [1940, 4

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  807 1782 1928]
 [   0    0    0 ... 1054 4265  629]
 [   0    0    0 ...   68 2648 4562]
 ...
 [   0    0    0 ... 3825 4974 3247]
 [   0    0    0 ... 3809 4952 4175]
 [   0    0    0 ... 4649   68 4999]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2182,
        901, 1893, 1604, 2118, 2230, 3082,  807, 1782, 1928])

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 20ms/step - loss: 0.3327 - accuracy: 0.8466 - val_loss: 0.1961 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 3s 17ms/step - loss: 0.1403 - accuracy: 0.9442 - val_loss: 0.1960 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0978 - accuracy: 0.9637 - val_loss: 0.2206 - val_accuracy: 0.9080
Epoch 4/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0708 - accuracy: 0.9751 - val_loss: 0.2384 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0462 - accuracy: 0.9856 - val_loss: 0.3239 - val_accuracy: 0.9120
Epoch 6/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0307 - accuracy: 0.9904 - val_loss: 0.3264 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0199 - accuracy: 0.9935 - val_loss: 0.4402 - val_accuracy: 0.9072

### Performance Metrics And Accuracy

In [30]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test,y_pred)

array([[3095,  324],
       [ 216, 2400]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9105219552609777

In [34]:
from tensorflow.keras.layers import Dropout
## Creating model with dropout
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])